In [1]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import config
import matplotlib.pyplot as plt

2023-12-04 17:32:09.738300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def resize_and_crop_image(image, size):
    target_ratio = size[0] / size[1]
    image_ratio = image.width / image.height

    if image_ratio > target_ratio:
        new_width = int(target_ratio * image.height)
        left = (image.width - new_width) / 2
        top = 0
        right = left + new_width
        bottom = image.height
    else:
        new_height = int(image.width / target_ratio)
        left = 0
        top = (image.height - new_height) / 2
        right = image.width
        bottom = top + new_height

    image = image.crop((left, top, right, bottom))
    image = image.resize(size, Image.LANCZOS)
    
    return image

In [4]:
def preprocess_image(image):
    processed_image = resize_and_crop_image(image, (700,700))
    processed_image = np.array(processed_image)
    processed_image = processed_image / 255.0
    processed_image = np.expand_dims(processed_image, axis=0)
    return processed_image

In [5]:
model = load_model('model_checkpoints/checkpoint_20-0.81.h5')

In [6]:
directories = [config.serving_set_dir_digital, config.serving_set_dir_film]
class_names = ['digital', 'film']

In [7]:
true_labels = []
predicted_labels = []

In [8]:
correct_predictions = 0
total_predictions = 0

In [9]:
for directory_path in directories:
    true_class = os.path.basename(directory_path)
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(directory_path, filename)
            image = Image.open(image_path)
            processed_image = preprocess_image(image)
            
            prediction = model.predict(processed_image)
            predicted_class = 'film' if prediction[0][0] > 0.5 else 'digital'

            true_labels.append(true_class)
            predicted_labels.append(predicted_class)

1/1 [==============================] - 0s 128ms/step


KeyboardInterrupt: 

In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy on test set: {accuracy:.2f}%")

In [ ]:
label_to_int = {name: index for index, name in enumerate(class_names)}
true_int_labels = [label_to_int[label] for label in true_labels]
predicted_int_labels = [label_to_int[label] for label in predicted_labels]

In [ ]:
cm = confusion_matrix(true_int_labels, predicted_int_labels)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()